In [1]:
import pandas as pd

## Importing MFT data

In [2]:
mft_df = pd.read_excel("MFT Returns by Zip 2019.xlsx", dtype={'Zip': object})

In [3]:
mft_df.columns

Index(['Zip', 'City', 'State', 'County', 'Org ID', 'United Way',
       '#e-filed returns', '#EITC returns', 'Average EITC Amount',
       'Average Refund Amount', 'Average Age'],
      dtype='object')

#### Dropping columns with sensistive data

In [4]:
mft_df.drop(axis='columns', labels=['#EITC returns', 'Average EITC Amount', 'Average Refund Amount', 'Average Age'], inplace=True)

#### Dropping unprocessable rows

In [16]:
print(mft_df[pd.isnull(mft_df['United Way'])].size); print(mft_df.size)

3619
125545


Dropping any row that has a letter in the zip code or doesn't have an associated United Way. Because we will be filling unmatched zip codes with the closest zip code served by the same United Way, we can't use rows without a United Way. Luckily, the rows without a United Way appear to mostly be the rows with nonexistent or malformed zip codes.

This removes 3,619 rows from the data set, out of 125,545 total rows.

In [17]:
mft_df = mft_df[~mft_df.Zip.str.match("[a-zA-Z]") & ~pd.isnull(mft_df['United Way'])]

In [19]:
mft_df[['Zip', '#e-filed returns']].to_csv("mft_returns_2019.csv", index=False)

### Importing ACS data

In [20]:
acs = pd.read_csv("ACS 2019_income_data_zip_accessible.csv", dtype={'ZIP': object})

### Merging ACS and MFT data

Right outer merge will get all rows from the MFT dataset and any rows from the ACS dataset that match.

In [21]:
merged = pd.merge(acs, mft_df, how="right", left_on="ZIP", right_on="Zip")

Sorting on zip because we'll be filling missing zip codes with their neighbors.

In [22]:
merged = merged.sort_values('Zip')

Forward filling missing zip codes. I.e. if we have two rows like

row | zip
----|------
1  | 48103
2  | NA

then the NA will become 48103.

In [24]:
merged["filled_zip"] = merged.groupby('United Way').apply(lambda group: group['ZIP'].fillna(method='ffill')).reset_index(level=0, drop="True")

Looking to see if there are any clusters we should be worried about. It mostly looks good - even when there are big clusters of zip codes, they appear to mainly be in the same city.

In [25]:
top5_filled_zips = merged[['filled_zip']].value_counts().head(5).reset_index()['filled_zip']

In [26]:
merged[merged.filled_zip.isin(top5_filled_zips)]

,ZIP,total,me_total,vita_eligible,vita_per_capita,id,Zip,City,State,County,Org ID,United Way,#e-filed returns,filled_zip
1804,11239,6705.0,193.0,5285.0,0.788218,8600000US11239,11239,Brooklyn,NY,Kings,34370F,United Way of New York City,21.0,11239
1805,NaN,NaN,NaN,NaN,NaN,NaN,11247,Brooklyn,NY,Kings,34370F,United Way of New York City,1.0,11239
1806,NaN,NaN,NaN,NaN,NaN,NaN,11249,Brooklyn,NY,Kings,34370F,United Way of New York City,31.0,11239
1807,NaN,NaN,NaN,NaN,NaN,NaN,11300,NaN,NaN,NaN,34370F,United Way of New York City,2.0,11239
1808,NaN,NaN,NaN,NaN,NaN,NaN,11312,NaN,NaN,NaN,34370F,United Way of New York City,1.0,11239
1809,NaN,NaN,NaN,NaN,NaN,NaN,11318,NaN,NaN,NaN,34370F,United Way of New York City,1.0,11239
1810,NaN,NaN,NaN,NaN,NaN,NaN,11347,NaN,NaN,NaN,34370F,United Way of New York City,1.0,11239
13609,77099,17017.0,553.0,11869.0,0.697479,8600000US77099,77099,Houston,TX,Harris,46377F,United Way of Greater Houston,6.0,77099
13610,NaN,NaN,NaN,NaN,NaN,NaN,77235,Houston,TX,Harris,46377F,United Way of Greater Houston,1.0,77099
13611,NaN,NaN,NaN,NaN,NaN,NaN,77238,Houston,TX,Harris,46377F,United Way of Greater Houston,2.0,77099


Aggregate multiple rows of zip codes into one, getting a sum of the # of e-files and the not-nil row values for everything else.

In [30]:
# sorting by ZIP instead of Zip makes sure we have the ACS row first
agg = merged.sort_values('ZIP').groupby('filled_zip').agg({'ZIP': 'first', 
                                                     'total': 'first',
                                                     'me_total': 'first',
                                                     'vita_eligible': 'first',
                                                     'vita_per_capita': 'first',
                                                     'id': 'first',
                                                     'City': 'first',
                                                     'State': 'first',
                                                     'County': 'first',
                                                     'Org ID': 'first',
                                                     'United Way': 'first',
                                                     '#e-filed returns': 'sum',
                                                     'filled_zip': 'first'})

In [31]:
agg[agg.filled_zip.isin(top5_filled_zips)]

,ZIP,total,me_total,vita_eligible,vita_per_capita,id,City,State,County,Org ID,United Way,#e-filed returns,filled_zip
filled_zip,,,,,,,,,,,,,
11239,11239,6705.0,193.0,5285.0,0.788218,8600000US11239,Brooklyn,NY,Kings,34370F,United Way of New York City,58.0,11239
77099,77099,17017.0,553.0,11869.0,0.697479,8600000US77099,Houston,TX,Harris,46377F,United Way of Greater Houston,13.0,77099
87144,87144,14444.0,431.0,5628.0,0.389643,8600000US87144,Rio Rancho,NM,Sandoval,33010F,United Way of Central New Mexico,64.0,87144
92155,92155,0.0,12.0,0.0,NaN,8600000US92155,San Diego,CA,San Diego,05405U,United Way of San Diego County,10.0,92155
93730,93730,4454.0,216.0,1018.0,0.228559,8600000US93730,Fresno,CA,Fresno,05105F,United Way of Fresno and Madera Counties,28.0,93730


In [32]:
agg.to_csv("ACS_MFT_2019_data.csv")